In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('Downloads/dataset-3.csv')
# Reset the index and move 'id_start' to a regular column
df.reset_index(inplace=True)

In [2]:
# Question 1: Distance Matrix Calculation
def calculate_distance_matrix(df):
    pivot_df = df.pivot_table(values='distance', index='id_start', columns='id_end', fill_value=0)
    distance_matrix = pivot_df + pivot_df.T
    return distance_matrix

In [3]:
result_distance_matrix = calculate_distance_matrix(df)
print(result_distance_matrix)

         1001400  1001402  1001404  1001406  1001408  1001410  1001412  \
1001400      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1001402      NaN      0.0     20.2      0.0      0.0      0.0      0.0   
1001404      NaN     20.2      0.0     16.0      0.0      0.0      0.0   
1001406      NaN      0.0     16.0      0.0     21.7      0.0      0.0   
1001408      NaN      0.0      0.0     21.7      0.0     11.1      0.0   
1001410      NaN      0.0      0.0      0.0     11.1      0.0     15.6   
1001412      NaN      0.0      0.0      0.0      0.0     15.6      0.0   
1001414      NaN      0.0      0.0      0.0      0.0      0.0     18.2   
1001416      NaN      0.0      0.0      0.0      0.0      0.0      0.0   
1001418      NaN      0.0      0.0      0.0      0.0      0.0      0.0   
1001420      NaN      0.0      0.0      0.0      0.0      0.0      0.0   
1001422      NaN      0.0      0.0      0.0      0.0      0.0      0.0   
1001424      NaN      0.0      0.0    

In [4]:
# Question 2: Unroll Distance Matrix
def unroll_distance_matrix(distance_matrix):
    distance_matrix_reset = distance_matrix.reset_index()
    unrolled_df = pd.melt(distance_matrix_reset, id_vars='index', var_name='id_end', value_name='distance')
    unrolled_df = unrolled_df[unrolled_df['index'] != unrolled_df['id_end']]
    unrolled_df = unrolled_df.rename(columns={'index': 'id_start'})
    return unrolled_df

In [5]:
result_unrolled_df = unroll_distance_matrix(result_distance_matrix)
print(result_unrolled_df.head())

   id_start   id_end  distance
1   1001402  1001400       NaN
2   1001404  1001400       NaN
3   1001406  1001400       NaN
4   1001408  1001400       NaN
5   1001410  1001400       NaN


In [6]:
# Question 3: Finding IDs within Percentage Threshold
def find_ids_within_ten_percentage_threshold(df, reference_value):
    reference_avg_distance = df[df['id_start'] == reference_value]['distance'].mean()
    lower_threshold = reference_avg_distance - (reference_avg_distance * 0.10)
    upper_threshold = reference_avg_distance + (reference_avg_distance * 0.10)
    result_list = df[(df['distance'] >= lower_threshold) & (df['distance'] <= upper_threshold)]['id_start'].unique()
    return sorted(result_list)

In [7]:
reference_value = 123
result_list = find_ids_within_ten_percentage_threshold(result_unrolled_df, reference_value)
print(result_list)

[]


In [8]:
# Question 4: Calculate Toll Rate
def calculate_toll_rate(df):
    rate_coefficients = {'moto': 0.8, 'car': 1.2, 'rv': 1.5, 'bus': 2.2, 'truck': 3.6}
    for vehicle in rate_coefficients:
        df[vehicle] = df['distance'] * rate_coefficients[vehicle]
    return df

In [9]:
result_with_toll_rate = calculate_toll_rate(result_unrolled_df)
print(result_with_toll_rate.head())

   id_start   id_end  distance  moto  car  rv  bus  truck
1   1001402  1001400       NaN   NaN  NaN NaN  NaN    NaN
2   1001404  1001400       NaN   NaN  NaN NaN  NaN    NaN
3   1001406  1001400       NaN   NaN  NaN NaN  NaN    NaN
4   1001408  1001400       NaN   NaN  NaN NaN  NaN    NaN
5   1001410  1001400       NaN   NaN  NaN NaN  NaN    NaN


In [10]:
# Question 5: Calculate Time-Based Toll Rates
def calculate_time_based_toll_rates(df):
    # Convert index to datetime
    df.index = pd.to_datetime(df.index)
    df['start_day'] = df['end_day'] = df['start_time'] = df['end_time'] = ''
    
    # Weekdays (Monday - Friday)
    df.loc[(df.index.weekday >= 0) & (df.index.weekday < 5), 'start_day'] = 'Monday'
    df.loc[(df.index.weekday >= 0) & (df.index.weekday < 5), 'end_day'] = 'Sunday'
    df.loc[(df.index.weekday >= 0) & (df.index.weekday < 5), 'start_time'] = pd.to_datetime('00:00:00').time()
    df.loc[(df.index.weekday >= 0) & (df.index.weekday < 5), 'end_time'] = pd.to_datetime('23:59:59').time()

    # Weekends (Saturday and Sunday)
    df.loc[(df.index.weekday >= 5), 'start_day'] = 'Saturday'
    df.loc[(df.index.weekday >= 5), 'end_day'] = 'Sunday'
    df.loc[(df.index.weekday >= 5), 'start_time'] = pd.to_datetime('00:00:00').time()
    df.loc[(df.index.weekday >= 5), 'end_time'] = pd.to_datetime('23:59:59').time()

    # Modify values of vehicle columns according to time ranges
    df.loc[(df.index.hour >= 0) & (df.index.hour < 10), ['moto', 'car', 'rv', 'bus', 'truck']] *= 0.8
    df.loc[(df.index.hour >= 10) & (df.index.hour < 18), ['moto', 'car', 'rv', 'bus', 'truck']] *= 1.2
    df.loc[(df.index.hour >= 18) & (df.index.hour <= 23), ['moto', 'car', 'rv', 'bus', 'truck']] *= 0.8
    df.loc[df.index.weekday >= 5, ['moto', 'car', 'rv', 'bus', 'truck']] *= 0.7

    return df

In [11]:
result_with_time_based_toll_rates = calculate_time_based_toll_rates(result_with_toll_rate)
print(result_with_time_based_toll_rates.head())

                               id_start   id_end  distance  moto  car  rv  \
1970-01-01 00:00:00.000000001   1001402  1001400       NaN   NaN  NaN NaN   
1970-01-01 00:00:00.000000002   1001404  1001400       NaN   NaN  NaN NaN   
1970-01-01 00:00:00.000000003   1001406  1001400       NaN   NaN  NaN NaN   
1970-01-01 00:00:00.000000004   1001408  1001400       NaN   NaN  NaN NaN   
1970-01-01 00:00:00.000000005   1001410  1001400       NaN   NaN  NaN NaN   

                               bus  truck start_day end_day start_time  \
1970-01-01 00:00:00.000000001  NaN    NaN    Monday  Sunday   00:00:00   
1970-01-01 00:00:00.000000002  NaN    NaN    Monday  Sunday   00:00:00   
1970-01-01 00:00:00.000000003  NaN    NaN    Monday  Sunday   00:00:00   
1970-01-01 00:00:00.000000004  NaN    NaN    Monday  Sunday   00:00:00   
1970-01-01 00:00:00.000000005  NaN    NaN    Monday  Sunday   00:00:00   

                               end_time  
1970-01-01 00:00:00.000000001  23:59:59  
1970-01-